<a href="https://colab.research.google.com/github/nnennandukwe/GenAI-Dev-Onboarding-Starter-Kit/blob/main/GenAI_Dev_Onboarding_Starter_Kit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gen AI Dev Onboarding Starter Kit #1: Intro to RAG! 🚀

Welcome! This notebook guides you through a Retrieval Augmented Generation (RAG) pipeline demonstration, built for developers (or enthusiasts) looking to get started in hands-on Generative AI!

### Tools You'll Use

- LangChain
- OpenAI
- ChromaDB
- Ragas


### Key steps covered:
1.  Setting up the environment and installing dependencies (including Langchain).
2.  Processing local documents (`doc1.txt` - Security Guidelines, `doc2.txt` - Company FAQs) using Langchain document loaders and text splitters.
3.  Generating embeddings using Langchain wrappers for OpenAI (`text-embedding-3-small`).
4.  Storing and indexing embeddings in ChromaDB using Langchain integration.
5.  Performing Question Answering using a Langchain `RetrievalQA` chain.
6.  Evaluating the RAG pipeline using Ragas (metrics: context precision, context recall, faithfulness), leveraging Langchain components.

### Before you begin

*   You will need an OpenAI API key.
*   The project files (including `embedding_processor_langchain.py`, `evaluation_langchain.py`, `doc1.txt`, `doc2.txt`, and `pyproject.toml`) should be accessible in your Colab environment (e.g., by cloning a Git repository or uploading them) once you've run Step 1 in order to complete the guide!

## 2. Setup Environment

In [1]:
# Clone the project repository if not already done
!git clone https://github.com/nnennandukwe/GenAI-Dev-Onboarding-Starter-Kit.git

# Change directory to the project folder
%cd GenAI-Dev-Onboarding-Starter-Kit

print("Environment setup started...")

Cloning into 'GenAI-Dev-Onboarding-Starter-Kit'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 55 (delta 23), reused 9 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (55/55), 57.79 KiB | 523.00 KiB/s, done.
Resolving deltas: 100% (23/23), done.
/content/GenAI-Dev-Onboarding-Starter-Kit
Environment setup started...


In [2]:
# Confirm you're in the correct directory before we begin installation
import os
project_root = os.getcwd()
print(project_root) # output below should be /content/GenAI-Dev-Onboarding-Starter-Kit/

/content/GenAI-Dev-Onboarding-Starter-Kit


### Install Poetry for managing Python dependencies

In [3]:
# Install Poetry
!curl -sSL https://install.python-poetry.org | python3 -

# Add Poetry to PATH for the current Colab session
import os
os.environ["PATH"] += ":" + os.path.expanduser("~/.local/bin")

!poetry --version
print("Poetry installed.")

Retrieving Poetry metadata

# Welcome to Poetry!

This will download and install the latest version of Poetry,
a dependency and package manager for Python.

It will add the `poetry` command to Poetry's bin directory, located at:

/root/.local/bin

You can uninstall at any time by executing this script with the --uninstall option,
and these changes will be reverted.

Installing Poetry (2.1.3)
Installing Poetry (2.1.3): Creating environment
Installing Poetry (2.1.3): Installing Poetry
Installing Poetry (2.1.3): Creating script
Installing Poetry (2.1.3): Done

Poetry (2.1.3) is installed now. Great!

To get started you need Poetry's bin directory (/root/.local/bin) in your `PATH`
environment variable.

Add `export PATH="/root/.local/bin:$PATH"` to your shell configuration file.

Alternatively, you can call Poetry explicitly with `/root/.local/bin/poetry`.

You can test that everything is set up by executing:

`poetry --version`

Poetry (version 2.1.3)
Poetry installed.


### Install project dependencies with Poetry

In [4]:
# This command reads pyproject.toml and installs all dependencies including Langchain.
!poetry install --no-root
print("Project dependencies (including Langchain) installed.")

Creating virtualenv my-rag-project-lsGMS1cD-py3.11 in /root/.cache/pypoetry/virtualenvs
Updating dependencies
Resolving dependencies... (12.7s)

Package operations: 128 installs, 0 updates, 0 removals

  - Installing certifi (2025.4.26): Pending...
  - Installing charset-normalizer (3.4.2): Pending...
  - Installing h11 (0.16.0): Pending...
  - Installing idna (3.10): Pending...
  - Installing sniffio (1.3.1): Pending...
  - Installing typing-extensions (4.13.2): Pending...
  - Installing typing-extensions (4.13.2): Pending...
  - Installing sniffio (1.3.1): Downloading... 0%
  - Installing typing-extensions (4.13.2): Pending...
  - Installing h11 (0.16.0): Pending...
  - Installing idna (3.10): Pending...
  - Installing sniffio (1.3.1): Downloading... 0%
  - Installing typing-extensions (4.13.2): Pending...
  - Installing charset-normalizer (3.4.2): Downloading... 0%
  - Installing h11 (0.16.0): Pending...
  - Installing idna (3.10): Pending...
  - Installing sniffio (1.3.1): Download

### Set up OpenAI API Key

In [5]:
import os
from google.colab import userdata
from getpass import getpass

api_key = os.environ.get("OPENAI_API_KEY")
colab_api_key = userdata.get('OPENAI_API_KEY')

if not api_key:

  if colab_api_key:
    api_key = colab_api_key
    os.environ["OPENAI_API_KEY"] = api_key
    print("Using Google Colab API key.")
  # Check for Google Colab key
  else:
    # Request for new OpenAI API key if none available
    api_key = getpass("Please enter your OpenAI API key: ")

    os.environ["OPENAI_API_KEY"] = api_key
    colab_api_key = api_key

if os.environ.get("OPENAI_API_KEY") or colab_api_key:
    print("OpenAI API key set successfully!")
else:
    print("Failed to set OpenAI API key.")

Using Google Colab API key.
OpenAI API key set successfully!


## 3. Data Processing and Embedding (Langchain)

Now, we will use the **Langchain-based script** (`embedding_processor_langchain.py`) to:
1. Load documents (`doc1.txt`, `doc2.txt`) using `TextLoader`.
2. Split documents into chunks using `RecursiveCharacterTextSplitter`.
3. Generate embeddings with `OpenAIEmbeddings` (`text-embedding-3-small`).
4. Store chunks and embeddings in ChromaDB via its Langchain integration.

> Make sure `doc1.txt`, `doc2.txt`, and `embedding_processor_langchain.py` (inside the `my_rag_project` subdirectory) are present.

### Double-check current directory and available script files

In [6]:
project_root = os.getcwd() # Should be /content/GenAI-Dev-Onboarding-Starter-Kit
script_dir = os.path.join(project_root, "my_rag_project") # where scripts go

!ls -l {project_root}
!ls -l {script_dir}

total 456
-rw-r--r-- 1 root root   9184 May 15 19:59 colab_notebook_guide.md
-rw-r--r-- 1 root root  10219 May 15 19:59 doc1.txt
-rw-r--r-- 1 root root   3634 May 15 19:59 doc2.txt
-rw-r--r-- 1 root root  11613 May 15 19:59 GenAI_Dev_Onboarding_Starter_Kit.ipynb
drwxr-xr-x 2 root root   4096 May 15 19:59 my_rag_project
-rw-r--r-- 1 root root 399471 May 15 20:01 poetry.lock
-rw-r--r-- 1 root root    518 May 15 19:59 pyproject.toml
-rw-r--r-- 1 root root   5474 May 15 19:59 README.md
drwxr-xr-x 3 root root   4096 May 15 19:59 src
drwxr-xr-x 2 root root   4096 May 15 19:59 tests
total 16
-rw-r--r-- 1 root root 4473 May 15 19:59 embedding_processor_langchain.py
-rw-r--r-- 1 root root 6404 May 15 19:59 evaluation_langchain.py


### Run the Langchain embedding processor script

In [ ]:
# This script will use Langchain for loading, chunking, embedding, and storing in ChromaDB.
# Ensure your OPENAI_API_KEY is set.

!poetry run python my_rag_project/embedding_processor_langchain.py

print("Langchain embedding process execution attempted.")
# Check for the new chroma_db_langchain directory
!ls -l
!ls -l chroma_db_langchain # This should exist if the script ran successfully

## 4. Ragas Evaluation (Langchain Pipeline)

Next, we'll use the **Langchain-based Ragas evaluation script** (`evaluation_langchain.py`). This script will:
1.  Connect to the ChromaDB populated by the Langchain embedding script.
2.  Set up a Langchain `RetrievalQA` chain using an OpenAI LLM (`gpt-3.5-turbo` or similar) and the ChromaDB retriever.
3.  Generate answers for predefined questions using the QA chain.
4.  Retrieve contexts (source documents) used by the QA chain.
5.  Calculate Ragas metrics: `context_precision`, `context_recall`, and `faithfulness`.

> Make sure `evaluation_langchain.py` (inside `my_rag_project` subdirectory) is present.

### Run the Langchain Ragas evaluation script

In [ ]:
# This script requires the ChromaDB (langchain version) to be populated.
# It also requires the OPENAI_API_KEY.
!poetry run python my_rag_project/evaluation_langchain.py

print("Langchain Ragas evaluation process execution attempted.")

## 5. Conclusion: You Did it! 🎊 ✅

This notebook demonstrated the core steps of setting up a RAG pipeline **using Langchain**, from document processing and embedding to QA and evaluation with Ragas.

**Further exploration:**
*   Explore the `embedding_processor_langchain.py` and `evaluation_langchain.py` files to get a deeper look into the embedding and evaluation code!
*   Try different LLMs available through Langchain. (all you have to do is edit the name of the model `LLM_MODEL` value in `evaluation_langchain.py` file!)
*   Explore more advanced Langchain chains and agents for RAG.
*   Expand the evaluation dataset and [other Ragas metrics](https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/) in conjunction with Langchain.